In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch
import numpy as np
from torch.utils.data import TensorDataset, Dataset, DataLoader
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import pandas as pd

train_df = pd.read_csv("data/kddcup99_train.csv",on_bad_lines='skip',header=None)
test_df = pd.read_csv("data/kddcup99_test.csv",on_bad_lines='skip',header=None)
attack_types = pd.read_table("data/trainning_attach_types", header=None,delim_whitespace=True)
attack_dict = {attack_types[0][i]+'.':1 for i in range(len(attack_types))}
attack_dict['normal.'] = 0
# attack_dict
train_df[41] = train_df[41].replace(attack_dict)
test_df[41] = test_df[41].replace(attack_dict)
map_dict = {}
for column in train_df.columns:
    if pd.api.types.is_object_dtype(train_df[column]):
        factorized_column, map = train_df[column].factorize()
        train_df[column] = factorized_column
        map_dict[column] = {map[i]:i for i in range(len(map))}

        _, map_t = test_df[column].factorize()
        # print([i for i in map_t if i not in map])
map_dict[2]['http_2784']=68
map_dict[2]['aol']=69
# map_dict
for column in test_df.columns:
    # print(column)
    if column in map_dict.keys():
        test_df[column] = test_df[column].replace(map_dict[column])
# test_df

train_features, train_labels = train_df.iloc[:,:-1], train_df.iloc[:,-1]
test_features, test_labels = test_df.iloc[:,:-1], test_df.iloc[:,-1]

train_features = train_features.to_numpy()
train_labels = train_labels.to_numpy()
test_features = test_features.to_numpy()
test_labels = test_labels.to_numpy()
tensor_x=torch.from_numpy(train_features.astype(np.float32))
tensor_y=torch.from_numpy(train_labels.astype(np.float32))
my_dataset=TensorDataset(tensor_x,tensor_y)
my_dataset_loader=DataLoader(my_dataset,batch_size=2000,shuffle=False)

In [3]:
test_df[41].describe()

count    978649.000000
mean          0.801111
std           0.399165
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: 41, dtype: float64

In [39]:
train_features, train_labels = train_df.iloc[:,:-1], train_df.iloc[:,-1]
test_features, test_labels = test_df.iloc[:,:-1], test_df.iloc[:,-1]

train_features = train_features.to_numpy()
train_labels = train_labels.to_numpy()
test_features = test_features.to_numpy()
test_labels = test_labels.to_numpy()
tensor_x=torch.from_numpy(train_features.astype(np.float32))
tensor_y=torch.from_numpy(train_labels.astype(np.float32))
my_dataset=TensorDataset(tensor_x,tensor_y)
my_dataset_loader=DataLoader(my_dataset,batch_size=2000,shuffle=False)

In [54]:
class mynet(nn.Module):
    def __init__(self,  act_layer = nn.ReLU(), init_method = nn.init.normal_):
        super(mynet, self).__init__()
        self.act_layer = act_layer
        self.init_method = init_method
        self.net = nn.Sequential(
            # [b, 41] => [b, rank]
            nn.Linear(41, 36),
            self.act_layer,
            nn.Linear(36, 24),
            self.act_layer,
            nn.Linear(24, 12),
            self.act_layer,
            nn.Linear(12, 6),
            self.act_layer,
            nn.Linear(6, 1),
            nn.Sigmoid(),
        )
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            self.init_method(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        batchsize = x.size(0)
        x = x.view(batchsize, -1)
        x = self.net(x)
        # reshape
        x = x.view(batchsize, -1)
 
        return x

In [31]:
def test_train(act_layer = nn.ReLU(), init_method = nn.init.normal_, optimizer = optim.Adam):
    epochs = 1
    lr = 1e-3
    model = mynet(act_layer, init_method)
    criterion = nn.MSELoss()
    optimizer = optimizer(model.parameters(), lr=lr)
    print(model)

    for epoch in range(epochs):
        print(epoch)
        total_loss = 0
        for batchidx, (x, y) in enumerate(my_dataset_loader):
            pred = model(x)
            loss=criterion(y.view(x.size(0),-1),pred)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss
        # if epoch % 10==0:
        #     print(total_loss.data.numpy())

        print(total_loss.data.numpy())
    
    return model

model = test_train(act_layer = nn.ReLU(), init_method = nn.init.normal_, optimizer = optim.Adam)

mynet(
  (act_layer): ReLU()
  (net): Sequential(
    (0): Linear(in_features=41, out_features=36, bias=True)
    (1): ReLU()
    (2): Linear(in_features=36, out_features=24, bias=True)
    (3): ReLU()
    (4): Linear(in_features=24, out_features=12, bias=True)
    (5): ReLU()
    (6): Linear(in_features=12, out_features=6, bias=True)
    (7): ReLU()
    (8): Linear(in_features=6, out_features=1, bias=True)
    (9): Sigmoid()
  )
)
0
246.24953


In [45]:
y_pred = model(torch.from_numpy(test_features.astype(np.float32))).detach().numpy()
y_pred[y_pred<0.5] = 0
y_pred[y_pred!=0] = 1
y_pred

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [43]:
test_labels

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [46]:
precision, recall, F1_score, _ = precision_recall_fscore_support(test_labels, y_pred, average=None)
acc = accuracy_score(test_labels, y_pred)
print("precision: {} \nrecall: {} \nF1 score: {} \naccuracy: {}".format(precision, recall, F1_score, acc))

precision: [0.96677173 0.99701689] 
recall: [0.98804992 0.99156894] 
F1 score: [0.97729502 0.99428545] 
accuracy: 0.9908690449793542


以上展示的是我们用 train(act_layer = nn.ReLU(), init_method = nn.init.normal_, optimizer = optim.Adam) 经过 1 个 epoch 得到的结果，

precision: [0.96677173 0.99701689] 
recall: [0.98804992 0.99156894] 
F1 score: [0.97729502 0.99428545] 
accuracy: 0.9908690449793542

已经可以简单用于说明结果，所以我们设置 epoch = 1 来进行 8 组组合的训练。

In [55]:
def train(act_layer = nn.ReLU(), init_method = nn.init.normal_, optimizer = optim.Adam):
    print(act_layer, init_method, optimizer)
    epochs = 1
    lr = 1e-3
    model = mynet(act_layer, init_method)
    criterion = nn.MSELoss()
    optimizer = optimizer(model.parameters(), lr=lr)
    # print(model)

    for epoch in range(epochs):
        # print(epoch)
        total_loss = 0
        for batchidx, (x, y) in enumerate(my_dataset_loader):
            pred = model(x)
            loss=criterion(y.view(x.size(0),-1),pred)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss
        # if epoch % 10==0:
        #     print(total_loss.data.numpy())

        # print(total_loss.data.numpy())
    
    y_pred = model(torch.from_numpy(test_features.astype(np.float32))).detach().numpy()
    y_pred[y_pred<0.5] = 0
    y_pred[y_pred!=0] = 1
    
    precision, recall, F1_score, _ = precision_recall_fscore_support(test_labels, y_pred, average=None)
    acc = accuracy_score(test_labels, y_pred)
    print("precision: {} \nrecall: {} \nF1 score: {} \naccuracy: {}".format(precision, recall, F1_score, acc))

    return model

for act_layer in (nn.ReLU(), nn.Sigmoid()):
    for init_method in (nn.init.normal_, nn.init.kaiming_normal_):
        for optimizer in (optim.Adam, optim.SGD):
            train(act_layer, init_method, optimizer)

ReLU() <function normal_ at 0x000001E40380E670> <class 'torch.optim.adam.Adam'>


c:\Users\Xavier\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision: [0.         0.80111051] 
recall: [0. 1.] 
F1 score: [0.         0.88957397] 
accuracy: 0.8011105105098968
ReLU() <function normal_ at 0x000001E40380E670> <class 'torch.optim.sgd.SGD'>
precision: [1.        0.8011187] 
recall: [5.13761091e-05 1.00000000e+00] 
F1 score: [1.02746939e-04 8.89579013e-01] 
accuracy: 0.8011207286780041
ReLU() <function kaiming_normal_ at 0x000001E40380ED30> <class 'torch.optim.adam.Adam'>
precision: [0.45945126 0.98782353] 
recall: [0.96433471 0.71832869] 
F1 score: [0.62237556 0.83179211] 
accuracy: 0.7672566977537401
ReLU() <function kaiming_normal_ at 0x000001E40380ED30> <class 'torch.optim.sgd.SGD'>
precision: [0.25309849 0.98640663] 
recall: [0.98453065 0.27868919] 
F1 score: [0.40267833 0.43459296] 
accuracy: 0.4190736413157322
Sigmoid() <function normal_ at 0x000001E40380E670> <class 'torch.optim.adam.Adam'>
precision: [0.97792252 0.99856805] 
recall: [0.99425615 0.99442734] 
F1 score: [0.9860217  0.99649339] 
accuracy: 0.9943932911595474
Si

c:\Users\Xavier\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


初步实验，结果显示在我们的网络中：

激活函数： Sigmoid 一般比 ReLU 效果更好，

初始化： Sigmoid + kaiming 效果好，ReLU 则是 normal 效果好

优化器：Adam 一般比 SGD 效果更好

最优的组合是 Sigmoid + kaiming + Adam, 达到很高的 p, r 以及最终正确率如下：

precision: [0.97660616 0.99959726] 

recall: [0.99838679 0.99406255] 

F1 score: [0.98737637 0.99682222] 

accuracy: 0.9949225922675035

全部结果：

ReLU() <function normal_ at 0x000001E40380E670> <class 'torch.optim.adam.Adam'>

precision: [0.         0.80111051] 

recall: [0. 1.] 

F1 score: [0.         0.88957397] 

accuracy: 0.8011105105098968

ReLU() <function normal_ at 0x000001E40380E670> <class 'torch.optim.sgd.SGD'>

precision: [1.        0.8011187] 

recall: [5.13761091e-05 1.00000000e+00] 

F1 score: [1.02746939e-04 8.89579013e-01] 

accuracy: 0.8011207286780041

ReLU() <function kaiming_normal_ at 0x000001E40380ED30> <class 'torch.optim.adam.Adam'>

precision: [0.45945126 0.98782353] 

recall: [0.96433471 0.71832869] 

F1 score: [0.62237556 0.83179211] 

accuracy: 0.7672566977537401

ReLU() <function kaiming_normal_ at 0x000001E40380ED30> <class 'torch.optim.sgd.SGD'>

precision: [0.25309849 0.98640663] 

recall: [0.98453065 0.27868919] 

F1 score: [0.40267833 0.43459296] 

accuracy: 0.4190736413157322

Sigmoid() <function normal_ at 0x000001E40380E670> <class 'torch.optim.adam.Adam'>

precision: [0.97792252 0.99856805] 

recall: [0.99425615 0.99442734] 

F1 score: [0.9860217  0.99649339] 

accuracy: 0.9943932911595474

Sigmoid() <function normal_ at 0x000001E40380E670> <class 'torch.optim.sgd.SGD'>

precision: [0.14175307 0.79268503] 

recall: [0.09159333 0.86232248] 

F1 score: [0.11128207 0.82603869] 

accuracy: 0.7090325540617729

Sigmoid() <function kaiming_normal_ at 0x000001E40380ED30> <class 'torch.optim.adam.Adam'>

precision: [0.97660616 0.99959726] 

recall: [0.99838679 0.99406255] 

F1 score: [0.98737637 0.99682222] 

accuracy: 0.9949225922675035

Sigmoid() <function kaiming_normal_ at 0x000001E40380ED30> <class 'torch.optim.sgd.SGD'>

precision: [0.         0.80111051] 

recall: [0. 1.] 

F1 score: [0.         0.88957397] 

accuracy: 0.8011105105098968